# Bibliotecas 

In [1]:
import sys
sys.path.append('/eos/user/t/thenriqu/Dark_Matter/LGBM_hhdm_analysis/')


import json
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

import hepherolib.analysis as ana
from lgbm.controllers_lgb import LGBLearner, LGBModel

ana.start()
plt.style.use("default")

# Configuração

In [2]:
period = '17'
year_style = 2017
dataset_year = "2017"
basedir = '/eos/user/t/thenriqu/Dark_Matter/Amostras/hhdmAnalysis_deepJet_Regions/datasets'
dataset_name = basedir.split('/')[-2]

# Data folder
data_path = f"./data/{dataset_name}/{dataset_year}"

# Setup output folders
models_path = f"./models/{dataset_name}/{dataset_year}"
Path(models_path).mkdir(parents=True, exist_ok=True)

# Lendo os dados

In [3]:
model_name = "LGB_multi_signal"

X_train = pd.read_csv(f"{data_path}/{model_name}-train-data.csv")
X_test = pd.read_csv(f"{data_path}/{model_name}-test-data.csv")

W_train = X_train.pop("modelWeight").values
W_test = X_test.pop("modelWeight").values

Y_train = X_train.pop("Label").values
Y_test = X_test.pop("Label").values

evtWeight_train = X_train.pop("evtWeight").values
evtWeight_test = X_test.pop("evtWeight").values

In [4]:
X_train.head()

,LeadingLep_pt,LepLep_deltaM,LepLep_deltaR,LepLep_pt,MET_LepLep_Mt,MET_LepLep_deltaPhi,MET_pt,MT2LL,Nbjets,TrailingLep_pt
0,47.163464,0.235672,2.265808,67.92354,128.03561,2.862613,61.526054,61.06800,1,24.065823
1,172.587650,1.318451,1.578457,175.15870,188.94249,2.127670,66.668750,65.81853,2,22.481535
2,126.914690,12.223457,0.707707,216.75772,221.90184,3.097569,56.819534,109.99913,2,99.981910
3,84.078545,1.770401,1.512148,108.57854,143.58008,2.310987,56.695393,70.79929,1,42.847990
4,143.305710,0.598106,0.690091,255.85414,313.49194,2.549555,104.960400,154.72617,1,128.369500


In [5]:
W_train

array([5.99839959e-01, 3.03238276e-01, 1.56494986e-03, ...,
       5.20275446e+00, 1.85627215e-01, 6.08263914e+00])

In [6]:
Y_train

array([0, 0, 0, ..., 1, 0, 1])

# Train

In [12]:
path1 = '/eos/user/t/thenriqu/Dark_Matter/LGBM_hhdm_analysis/Analysis_Regions/Analysis_2017/models/hhdmAnalysis_deepJet_Regions/2017/'


In [13]:
with open(path1 + 'LGB_multi_signal-bst-params.json', 'r') as f:
    best_params = json.load(f)

print(best_params)

{'subsample': 0.9, 'n_estimators': 9000, 'min_child_weight': 6, 'min_child_samples': 100, 'max_depth': 9, 'learning_rate': 0.008, 'eval_metric': 'error', 'colsample_bytree': 0.8}


In [14]:
params = { 'subsample': 0.9,
          'n_estimators': 9000,
          'min_child_weight': 6,
          'min_child_samples': 100,
          'max_depth': 9,
          'learning_rate': 0.008,
          'eval_metric': 'error',
          'colsample_bytree': 0.8
}

# Pode se ler através do arquivos .json salvo no código 2_find_hyperparams_lgb.ipynb
# params = json.load(open(f"{models_path}/{signal_name}-bst-params.json"))

num_boost_round = params.pop('n_estimators')
features = ["LeadingLep_pt", "LepLep_pt", "LepLep_deltaR", "LepLep_deltaM", "MET_pt", "MET_LepLep_Mt", "MET_LepLep_deltaPhi", "TrailingLep_pt", "MT2LL", "Nbjets"]

learner = LGBLearner(X_train, Y_train, W_train, features)
learner.train(params, num_boost_round)
learner.save_model(f"{models_path}/{model_name}-clf.model")
# learner.dump_model(f"{models_path}/{model_name}-raw.model")

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.107130 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2303
[LightGBM] [Info] Number of data points in the train set: 2091463, number of used features: 10
[LightGBM] [Info] Start training from score 0.666667


# Nofiticar quando o Notebook terminar

In [15]:
from IPython.display import clear_output, display, HTML, Javascript

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Processo Completo!";
  window.speechSynthesis.speak(msg);
  alert("Processo Completo!")
"""))

<IPython.core.display.Javascript object>